# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [2]:
!pip install keras==2.3.1

     |████████████████████████████████| 377 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 16.9 MB/s eta 0:00:01


In [3]:
import datetime
import keras
#from keras.datasets import mnist
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D
#from keras import backend as K
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

Using TensorFlow backend.


In [4]:
#used to help some of the timing functions
now = datetime.datetime.now

In [5]:
# set some parameters
batch_size = 128 # size of the batch of data used
num_classes = 5 # number of classes within the data
epochs = 5 # number of times we rung through the data

In [6]:
# set some more parameters
img_rows, img_cols = 28, 28 # setting the pixel number for rows and cols
filters = 32 # the depth of each one of our next layers in the convo neural net
pool_size = 2 # reduces the image size by mapping a patch of pixels to a single value
kernel_size = 3 # grid size with weights overlayed onto the image

In [7]:
## This just handles some variability in how the input data is loaded
# using rgb would be (3,28,28), note ours is greyscale so only 1 and we have channel last
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [8]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

#train has features and y
# x_train takes first value of tuple, reshape to same rows to our input shape (60000, 28, 28, 1) for example
# making sure all values between 0 and 1

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1, #verbose 0 results in steps showing training at each epoch to be removed
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [9]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11493376/11490434 [==============================] - 0s 0us/step


In [10]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [11]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [12]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [13]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [14]:
# Now, let's train our model on the digits 5,6,7,8,9
# using the function we developed

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 52s 225ms/step - loss: 1.6278 - accuracy: 0.1698 - val_loss: 1.6020 - val_accuracy: 0.2380
Epoch 2/5
230/230 [==============================] - 51s 224ms/step - loss: 1.6009 - accuracy: 0.2300 - val_loss: 1.5721 - val_accuracy: 0.3769
Epoch 3/5
230/230 [==============================] - 51s 221ms/step - loss: 1.5761 - accuracy: 0.2860 - val_loss: 1.5418 - val_accuracy: 0.4625
Epoch 4/5
230/230 [==============================] - 52s 226ms/step - loss: 1.5453 - accuracy: 0.3540 - val_loss: 1.5099 - val_accuracy: 0.5445
Epoch 5/5
230/230 [==============================] - 52s 225ms/step - loss: 1.5205 - accuracy: 0.4007 - val_loss: 1.4750 - val_accuracy: 0.6106
Training time: 0:04:18.683933
Test score: 1.4749772548675537
Test accuracy: 0.610573947429657


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [15]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False
#so it tells keras in the CNN that it wants to freeze the layers

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [16]:
model.summary()
# see the total # of prarams at 600k but trainable is less due to the fact we are freezing the earlier layers

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [17]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 17s 69ms/step - loss: 1.5972 - accuracy: 0.2484 - val_loss: 1.5486 - val_accuracy: 0.3964
Epoch 2/5
240/240 [==============================] - 16s 68ms/step - loss: 1.5507 - accuracy: 0.3351 - val_loss: 1.4972 - val_accuracy: 0.4876
Epoch 3/5
240/240 [==============================] - 16s 67ms/step - loss: 1.5026 - accuracy: 0.4319 - val_loss: 1.4477 - val_accuracy: 0.6118
Epoch 4/5
240/240 [==============================] - 16s 66ms/step - loss: 1.4570 - accuracy: 0.5138 - val_loss: 1.3987 - val_accuracy: 0.7198
Epoch 5/5
240/240 [==============================] - 16s 66ms/step - loss: 1.4120 - accuracy: 0.5752 - val_loss: 1.3502 - val_accuracy: 0.7821
Training time: 0:01:20.969474
Test score: 1.3502193689346313
Test accuracy: 0.7820587754249573


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [18]:
# Create layers and define the model as above
# so training on two separate datasets lt5 and gt 5
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [19]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 53s 221ms/step - loss: 1.6034 - accuracy: 0.2393 - val_loss: 1.5555 - val_accuracy: 0.4695
Epoch 2/5
240/240 [==============================] - 52s 216ms/step - loss: 1.5527 - accuracy: 0.3558 - val_loss: 1.4994 - val_accuracy: 0.7159
Epoch 3/5
240/240 [==============================] - 53s 221ms/step - loss: 1.5026 - accuracy: 0.4693 - val_loss: 1.4387 - val_accuracy: 0.7935
Epoch 4/5
240/240 [==============================] - 54s 223ms/step - loss: 1.4452 - accuracy: 0.5648 - val_loss: 1.3692 - val_accuracy: 0.8284
Epoch 5/5
240/240 [==============================] - 53s 221ms/step - loss: 1.3800 - accuracy: 0.6310 - val_loss: 1.2882 - val_accuracy: 0.8579
Training time: 0:04:24.731901
Test score: 1.288170337677002
Test accuracy: 0.8579490184783936


In [20]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [22]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)
#by flipping what we trained first before we got slightly worse results which defines that it is an art note we could have added epoch and been faster.

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 16s 69ms/step - loss: 1.5869 - accuracy: 0.2892 - val_loss: 1.5374 - val_accuracy: 0.3950
Epoch 2/5
230/230 [==============================] - 16s 69ms/step - loss: 1.5458 - accuracy: 0.3339 - val_loss: 1.4937 - val_accuracy: 0.4384
Epoch 3/5
230/230 [==============================] - 16s 68ms/step - loss: 1.5062 - accuracy: 0.3844 - val_loss: 1.4510 - val_accuracy: 0.5087
Epoch 4/5
230/230 [==============================] - 16s 68ms/step - loss: 1.4682 - accuracy: 0.4327 - val_loss: 1.4096 - val_accuracy: 0.5986
Epoch 5/5
230/230 [==============================] - 16s 67ms/step - loss: 1.4280 - accuracy: 0.4910 - val_loss: 1.3691 - val_accuracy: 0.6686
Training time: 0:01:18.843190
Test score: 1.3690930604934692
Test accuracy: 0.6685867309570312


---
### Machine Learning Foundation (C) 2020 IBM Corporation